In [1]:
import pandas as pd
import mne
import numpy as np
from datetime import timedelta
import Helper
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
FMIN = 2.
FMAX = 30.
SAMPLING_RATE = 200 # eeg sample rate
TIME_OFFSET = 0

delta_band = [0.5, 4]
theta_band = [4, 8]
alpha_band = [8,14]
beta_band = [14,30]
gamma_band = [30, 58]

all_ch = ['ch1','ch2','ch3','ch4']
# PICO_CHANNELS_ORDER = ['Pz', 'FCz', 'FC3', 'Oz']
PICO_CHANNELS_ORDER = ['FC3', 'FCz', 'Pz', 'Oz']

In [3]:
def filterEeg(eeg_data):
    for ch in all_ch:
        eeg_data[ch] = mne.filter.filter_data(eeg_data[ch].astype(float).to_numpy(), sfreq=SAMPLING_RATE, l_freq=FMIN, h_freq=FMAX, verbose=False)
        eeg_data[ch] = mne.filter.notch_filter(eeg_data[ch].astype(float).to_numpy(), Fs=SAMPLING_RATE, freqs=np.arange(60, 100, 60), verbose=False, filter_length='auto', phase='zero')

    return eeg_data

In [4]:
# Data

def changeTimeOffset(timestampSeries):
    timestampSeries = pd.to_datetime(timestampSeries)
    timestampSeries = timestampSeries.apply(lambda x : x - timedelta(days=0, seconds=0, microseconds=0, milliseconds=TIME_OFFSET, minutes=0, hours=0, weeks=0))
    return timestampSeries

def cleanEegData(eegData):
    cleanCols = ['ch1','ch2','ch3','ch4', "Timestamp"]
    cleanEegData = eegData[cleanCols]

    cleanEegData['Timestamp'] = changeTimeOffset(cleanEegData['Timestamp'])

    return cleanEegData

In [5]:
delta_band = [0.5, 4]
theta_band = [4, 8]
alpha_band = [8,14]
beta_band = [14,30]
gamma_band = [30, 58]

In [10]:
def getColumns():
    eegBands = ['delta', 'theta', 'alpha', 'beta', 'gamma']
    cols = []
    for ch in PICO_CHANNELS_ORDER:
        for eegBand in eegBands:
            cols.append(f"{ch}_{eegBand}")

    return cols

def calEegFeatures(eeg_data):
    eegFeature_df = pd.DataFrame(columns=getColumns())
    # band power
    for ch, ach in zip(all_ch, PICO_CHANNELS_ORDER):
        # band (改動了資料格式，加了.to_numpy())
        delta_bandpower = Helper.bandpower(eeg_data[ch].to_numpy(), SAMPLING_RATE, delta_band, window_sec = len(eeg_data[ch])/SAMPLING_RATE, relative=True)
        theta_bandpower = Helper.bandpower(eeg_data[ch].to_numpy(), SAMPLING_RATE, theta_band, window_sec = len(eeg_data[ch])/SAMPLING_RATE, relative=True)
        alpha_bandpower = Helper.bandpower(eeg_data[ch].to_numpy(), SAMPLING_RATE, alpha_band, window_sec = len(eeg_data[ch])/SAMPLING_RATE, relative=True)
        beta_bandpower = Helper.bandpower(eeg_data[ch].to_numpy(), SAMPLING_RATE, beta_band, window_sec = len(eeg_data[ch])/SAMPLING_RATE, relative=True)
        gamma_bandpower = Helper.bandpower(eeg_data[ch].to_numpy(), SAMPLING_RATE, gamma_band, window_sec = len(eeg_data[ch])/SAMPLING_RATE, relative=True)

        eegFeature_df[f'{ach}_delta'] = [delta_bandpower]
        eegFeature_df[f'{ach}_theta'] = [theta_bandpower]
        eegFeature_df[f'{ach}_alpha'] = [alpha_bandpower]
        eegFeature_df[f'{ach}_beta'] = [beta_bandpower]
        eegFeature_df[f'{ach}_gamma'] = [gamma_bandpower]

    return eegFeature_df


In [7]:
def set_label(dataInfo):
    addictLabel = int(dataInfo[0][0] == 'A')

    if dataInfo[3] == "PreTest":
        state = 0
    elif dataInfo[3] == "VRTest":
        state = 1
    else:
        state = 2

    stateLabel = state

    # label = {'addictLabel' : addictLabel, 'stateLabel':stateLabel, 'trainingCount': trainingCount}
    label = {'addictLabel' : addictLabel, 'stateLabel': stateLabel}

    return label

# Processing input

In [ ]:
# columns = ['Timestamp', 'ch1', 'ch2', 'ch3', 'ch4']
# csv_file = "NormalData/test08/VR_1_PostTest_EGG.csv"
# eeg_data = pd.read_csv(csv_file, header = None, names = columns)
# # print(eeg_data[0:5])
# eeg_data = cleanEegData(eeg_data)
# # print(eeg_data[0:5])
# eeg_data = filterEeg(eeg_data)

# result = pd.DataFrame(columns=getColumns())

# EEGfeatures = calEegFeatures(eeg_data)
# result = pd.concat([result, EEGfeatures], axis = 0)

In [19]:
workDir = r'C:\Users\User\Desktop\lab\Relaxing_BC&AC\分析'
os.chdir(workDir)
# inputPath = "Data/EEG/"
inputPath = "DataBC&AC/BC/EEG/"
data_name = ['data_name']
lable_names = ['addictLabel', 'stateLabel']
columns = ['Timestamp', 'ch1', 'ch2', 'ch3', 'ch4']

dataList = os.listdir(inputPath)
name = pd.DataFrame(columns = data_name)
result = pd.DataFrame(columns=getColumns())
lable = pd.DataFrame(columns=lable_names)

for dataName in dataList:
    # A1_VR_1_PreTest_EEG
    dataInfo = dataName[:-4].split('_')
    # if dataInfo[0] == 'A14' or dataInfo[0] == 'A17':
    #     continue

    eeg_data = pd.read_csv("%s%s"%(inputPath, dataName), header = None, names=columns)
    # print(eeg_data[0:5])
    eeg_data = cleanEegData(eeg_data)
    eeg_data = filterEeg(eeg_data)
    # print(eeg_data[0:5])

    # if (dataInfo[0] == 'N01' or dataInfo[0] == 'N02' or dataInfo[0] == 'N03' or dataInfo[0] == 'N04' or dataInfo[0] == 'N06' or dataInfo[0] == 'N08'
    #     or dataInfo[0] == 'A01' or dataInfo[0] == 'A02'):
    #     SAMPLING_RATE = 60 # eeg sample rate
    # else:
    #     SAMPLING_RATE = 200 # eeg sample rate

    if (dataInfo[3] == 'PreTest'):
        # print(eeg_data.iloc[0].Timestamp, eeg_data.iloc[0].Timestamp+timedelta(seconds=0.5))
        eeg_basedata = eeg_data[eeg_data.Timestamp <= eeg_data.iloc[0].Timestamp + timedelta(seconds=1.5)] # 1s 有些 baseline 會為0(sample rate = 60)
        # print(eeg_basedata.iloc[-1])
        eeg_data = eeg_data[eeg_data.Timestamp > eeg_data.iloc[0].Timestamp + timedelta(seconds=1.5)]

        EEGbasefeatures = calEegFeatures(eeg_basedata)
        datanameDF = pd.DataFrame({"data_name" : ["%s%s_%s"%(dataInfo[0], dataInfo[1], 'baseline')]})
        name = pd.concat([name, datanameDF], axis=0, ignore_index=True)
        result = pd.concat([result, pd.DataFrame(EEGbasefeatures, index=[0])], axis=0, ignore_index=True)
        lable = pd.concat([lable, pd.DataFrame({'addictLabel' : int(dataInfo[0][0] == 'A'), 'stateLabel': 3}, index=[0])], axis=0, ignore_index=True)

    EEGfeatures = calEegFeatures(eeg_data)

    # features = feature_Cal(data, sampleRate, 'RESdata')
    datanameDF = pd.DataFrame({"data_name" : ["%s%s_%s"%(dataInfo[0], dataInfo[1], dataInfo[3][:-4])]})
    name = pd.concat([name, datanameDF], axis=0, ignore_index=True)
    result = pd.concat([result, pd.DataFrame(EEGfeatures, index=[0])], axis=0, ignore_index=True)
    lable = pd.concat([lable, pd.DataFrame(set_label(dataInfo), index=[0])], axis=0, ignore_index=True)

result = name.join(result)
result = result.join(lable)


C:\Users\User\AppData\Local\Temp\ipykernel_14448\4167316373.py:49: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = pd.concat([result, pd.DataFrame(EEGfeatures, index=[0])], axis=0, ignore_index=True)


In [20]:
result.to_csv('FeatureBC&AC/BC/EEG_t_test.csv')
result

,data_name,FC3_delta,FC3_theta,FC3_alpha,FC3_beta,FC3_gamma,FCz_delta,FCz_theta,FCz_alpha,FCz_beta,...,Pz_alpha,Pz_beta,Pz_gamma,Oz_delta,Oz_theta,Oz_alpha,Oz_beta,Oz_gamma,addictLabel,stateLabel
0,A107VR_Post,0.365662,0.181453,0.279727,0.148546,0.020064,0.432946,0.254718,0.198922,0.099474,...,0.154548,0.116647,0.019485,0.301331,0.127778,0.400123,0.139830,0.026003,1,2
1,A107VR_baseline,0.260967,0.101071,0.128344,0.297420,0.019757,0.233505,0.048813,0.142012,0.436475,...,0.099790,0.340923,0.034411,0.755066,0.105640,0.057296,0.078943,0.019889,1,3
2,A107VR_Pre,0.731745,0.124683,0.054630,0.066921,0.008549,0.716811,0.154072,0.054812,0.058181,...,0.056501,0.069931,0.013303,0.760144,0.162146,0.042631,0.007786,0.000508,1,0
3,A10VR_Post,0.508603,0.158046,0.243986,0.078969,0.003648,0.576341,0.150562,0.188804,0.072837,...,0.198450,0.110626,0.009568,0.373312,0.109852,0.425104,0.081704,0.003692,1,2
4,A10VR_baseline,0.736730,0.067464,0.037311,0.054252,0.006233,0.838472,0.026080,0.013969,0.040256,...,0.027190,0.072403,0.016252,0.606352,0.054632,0.022401,0.114589,0.014624,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,A7VR_baseline,0.540183,0.131221,0.071768,0.130700,0.025825,0.627732,0.139593,0.100741,0.088706,...,0.249675,0.185957,0.019436,0.502371,0.057268,0.071684,0.063163,0.013634,1,3
113,A7VR_Pre,0.689244,0.109561,0.081223,0.116226,0.012938,0.589557,0.124141,0.108720,0.155698,...,0.128521,0.252460,0.037560,0.765811,0.062134,0.049746,0.076889,0.011528,1,0
114,A9VR_Post,0.457449,0.164228,0.260691,0.105306,0.008045,0.414285,0.284047,0.195715,0.096798,...,0.171870,0.169702,0.019868,0.246862,0.148254,0.490089,0.103101,0.010029,1,2
115,A9VR_baseline,0.649476,0.218323,0.079170,0.015479,0.000481,0.695309,0.146030,0.031001,0.025983,...,0.026873,0.014034,0.000858,0.360626,0.088895,0.170821,0.280788,0.023426,1,3


In [21]:
baselineData = result[result.stateLabel == 3]
EEGfData = result[result.stateLabel != 3]
# print(baselineData.shape)
# print(EEGfData.shape)
# print(baselineData[0:5])
# print("-------------------------------")
# print(EEGfData[0:5])
for index, row in EEGfData.iterrows():
    # print(row.data_name)
    dn = row.data_name.split('_')
    # print(dn[0])
    # print(baselineData.data_name.str.startswith(dn[0]))
    bld = baselineData[baselineData.data_name.str.startswith(dn[0])]
    # print(bld)
    eegFeature_df = pd.DataFrame(columns=getColumns())
    for c in eegFeature_df:
        row[c] = (row[c] - bld[c]) / bld[c] * 100
        # print(row[c])
        EEGfData.at[index, c] = row[c].iloc[0]
    # print(row)
    # print('______________________')
EEGfData.to_csv('FeatureBC&AC/BC/EEG_PCR_test.csv')
EEGfData

,data_name,FC3_delta,FC3_theta,FC3_alpha,FC3_beta,FC3_gamma,FCz_delta,FCz_theta,FCz_alpha,FCz_beta,...,Pz_alpha,Pz_beta,Pz_gamma,Oz_delta,Oz_theta,Oz_alpha,Oz_beta,Oz_gamma,addictLabel,stateLabel
0,A107VR_Post,40.118298,79.529821,117.950758,-50.055212,1.549874,85.411571,421.825887,40.074094,-77.209798,...,54.872976,-65.784892,-43.375005,-60.092063,20.955991,598.340583,77.127257,30.738237,1,2
2,A107VR_Pre,180.397834,23.360989,-57.434973,-77.499657,-56.729681,206.978437,215.637133,-61.402847,-86.670363,...,-43.380705,-79.487767,-61.340430,0.672554,53.488653,-25.595386,-90.137600,-97.447061,1,0
3,A10VR_Post,-30.964709,134.265668,553.932966,45.560133,-41.467240,-31.262971,477.297893,1251.555787,80.934980,...,629.851790,52.792677,-41.127339,-38.433205,101.077315,1797.695328,-28.698682,-74.750752,1,2
5,A10VR_Pre,8.962656,42.685907,-5.105437,-24.451855,-35.494372,-3.899204,175.885712,214.137008,26.486894,...,75.671924,-6.204544,-47.251913,32.810529,32.289862,69.625874,-50.324787,-50.993816,1,0
6,A111VR_Post,-6.458152,83.260814,-40.673032,-86.373392,-88.799996,90.521587,-56.512625,86.449411,-46.031274,...,-24.202849,-77.046120,-94.260839,40.358367,-63.053730,-2.540389,-68.319504,-75.957165,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,A6VR_Pre,471.688696,-70.785001,-50.745743,-65.078173,-73.753936,113.354328,-18.630044,-55.285738,5.015603,...,-60.274059,-78.313770,-58.954879,25.520673,-67.352890,0.059277,-65.041727,-87.567616,1,0
111,A7VR_Post,-9.456959,8.860496,296.316904,-44.593844,-87.409503,-22.334276,36.353589,119.877177,-10.527461,...,-18.353446,24.572007,64.892383,-49.794600,136.989504,656.348104,16.967420,-75.539653,1,2
113,A7VR_Pre,27.594533,-16.506231,13.174184,-11.074696,-49.900940,-6.081511,-11.068817,7.919703,75.522148,...,-48.524692,35.762786,93.245553,52.439369,8.498027,-30.603473,21.730949,-15.448813,1,0
114,A9VR_Post,-29.566451,-24.777544,229.280569,580.315220,1573.276897,-40.417221,94.512983,531.324305,272.543243,...,539.565626,1109.190332,2216.907357,-31.546316,66.775201,186.902160,-63.281540,-57.188098,1,2


In [16]:
EEGfData[0:1].data_name.str[-6]

0    V
Name: data_name, dtype: object

In [ ]:
# preResult = result[result['data_name'].str.endswith('_Pre')].reset_index(drop=True)
# # preResult.to_csv('Feature/EEG_PreF.csv')
# postResult = result[result['data_name'].str.endswith('_Post')].reset_index(drop=True)
# # postResult
# ppResult = pd.concat([preResult, postResult], axis=0, ignore_index=True)
# ppResult
# # postResult.to_csv('Feature/EEG_PostF.csv')
# ppResult.to_csv('Feature/EEG_PrePost.csv')